In [1]:
import pandas as pd 
import numpy as np


In [25]:
df = pd.read_excel('../data/processed/DOE_Combined.xlsx')

In [26]:
df.head()

,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
0,January,2005-01-04 00:00:00,6:00 p.m.,2005-01-14 00:00:00,12:00 p.m.,Eastern one third of the state of Kansas,SPP,Unknown,Winter Storm,211000,200
1,January,2005-01-05 00:00:00,4:00 p.m.,2005-01-13 00:00:00,6:00 p.m.,Akron and Mansfield areas,ECAR,Unknown,Ice Storm,246990,250
2,January,2005-01-05 00:00:00,9:10 p.m.,2005-01-16 00:00:00,11:00 a.m.,Indiana Michigan Region - Muncie District,ECAR,Unknown,Winter Ice Storm,114791,545
3,January,2005-01-07 00:00:00,1:00 p.m.,2005-10-01 00:00:00,8:00 a.m.,Northern California,WECC,Unknown,Winter Storm,442000,120
4,January,1/19/2005,9:17 a.m.,2005-01-19 00:00:00,9:27 a.m.,Island of Puerto Rico,PR,Unknown,Voltage Reduction,NaN,209


In [28]:
df.dtypes

Month                           object
Date Event Began                object
Time Event Began                object
Date of Restoration             object
Time of Restoration             object
Area Affected                   object
NERC Region                     object
Alert Criteria                  object
Event Type                      object
Demand Loss (MW)                object
Number of Customers Affected    object
dtype: object